In [1]:
script = """
#!/bin/bash

set -e

echo "Installing SSH Pass"

sudo apt-get install sshpass

echo "Configuring brokers"

CLUSTER_SSH_ADDRESS="{CLUSTER_SSH_ADDRESS}" 
CLUSTER_SSH_USER="{CLUSTER_SSH_USER}"
CLUSTER_SSH_PASSWORD="{CLUSTER_SSH_PASSWORD}"

BROKERS=("{BROKER_1_IP_ADDRESS}:9092" \
"{BROKER_2_IP_ADDRESS}:9092" \
"{BROKER_3_IP_ADDRESS}:9092" \
"{BROKER_4_IP_ADDRESS}:9092" )

NUM_BROKERS=${#BROKERS[@]}
NUM_BROKERS_HALF=$(($NUM_BROKERS/2))

for (( i=0; i<$NUM_BROKERS; i++ ))
do

THIS_BROKER_HOST=${BROKERS[i]}

echo "Forwarding $THIS_BROKER_HOST"

# register local interface
ip add a dev lo $(echo $THIS_BROKER_HOST | cut -d':' -f 1)

TUNNEL_PORT=22
if [ "$i" -ge "$NUM_BROKERS_HALF" ]; then
  # for load balancing between nodes
  TUNNEL_PORT=23
fi

# set up ssh tunneling with the broker
sshpass -p $CLUSTER_SSH_PASSWORD ssh -fnNT -o StrictHostKeyChecking=no -p $TUNNEL_PORT -L $THIS_BROKER_HOST:$THIS_BROKER_HOST $CLUSTER_SSH_USER@$CLUSTER_SSH_ADDRESS

done

sudo apt-get install -y python3-pip
sudo apt-get install -y libjasper-dev

pip3 install Pillow
pip3 install xmltodict
pip3 install EasyDict
pip3 install scikit-learn
pip3 install scikit-image
pip3 install opencv-python
pip3 install --upgrade pandas

echo "Done."
"""
dbutils.fs.put("dbfs:/databricks/init/{REPLACE_WITH_CLUSTER_NAME}/configure-kafka-access.sh", script, True)


print ("Init scripts installed")


In [2]:
dbutils.fs.rm("dbfs:/databricks/init/ntedemodbrclusterdemoscript/install-cntk.sh")